In [ ]:
model_id = "Qwen/Qwen2.5-7B-Instruct"
load_dict_path = "~/downloads/EAGLE-Qwen2.5-7B-Instruct/pytorch_model.bin"
save_dict_dir = "~/checkpoints/EAGLE-Qwen2.5-7B-Instruct/"

/home/scott/envs/subspec/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.85it/s]


Convert weights

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from specdecodes.models.draft_models.eagle_sd import EagleSDDraftModel

target_model = AutoModelForCausalLM.from_pretrained("model_id", device_map='meta')
draft_model = EagleSDDraftModel(target_model=target_model)

model_state_dict = torch.load(load_dict_path, map_location='cpu')
new_state_dict = {}
for key, value in model_state_dict.items():
    if key.startswith('layers.'):
        # add models prefix to the key
        new_key = f'model.{key}'
        new_state_dict[new_key] = value
    elif key.startswith("fc"):
        # add models prefix to the key
        new_key = f'fusion.{key}'
        new_state_dict[new_key] = value
    elif key.startswith('embed'):
        # skip the embedding layers
        continue
    else:
        new_state_dict[key] = value
model_state_dict = new_state_dict

draft_model.load_state_dict(model_state_dict)

<All keys matched successfully>

Save weights to directory

In [ ]:
import os
from safetensors.torch import save_model

# create folder if not exists
if not os.path.exists(save_dict_dir):
    os.makedirs(save_dict_dir)

save_model(draft_model, os.path.join(save_dict_dir, 'model.safetensors'))